In [6]:
pip install mit-news-classify

     |████████████████████████████████| 24.2 MB 12.0 MB/s 
     |████████████████████████████████| 2.1 MB 10.8 MB/s 
     |████████████████████████████████| 394.4 MB 10.2 MB/s 
     |████████████████████████████████| 75 kB 8.1 MB/s 
     |████████████████████████████████| 804.1 MB 16 kB/s 
     |████████████████████████████████| 56 kB 7.5 MB/s 
     |████████████████████████████████| 42 kB 3.7 MB/s 
     |████████████████████████████████| 14.9 MB 12.1 MB/s 
     |████████████████████████████████| 65 kB 8.1 MB/s 
     |████████████████████████████████| 6.0 MB 10.7 MB/s 
     |████████████████████████████████| 129 kB 11.2 MB/s 
     |████████████████████████████████| 1.0 MB 11.0 MB/s 
  Using cached typing_extensions-3.7.4.3-py3-none-any.whl (22 kB)
     |████████████████████████████████| 462 kB 10.8 MB/s 
  Using cached wrapt-1.12.1-cp38-cp38-linux_x86_64.whl
     |████████████████████████████████| 57 kB 7.7 MB/s 
     |████████████████████████████████| 3.8 MB 10.8 MB/s 
     |█████████

In [7]:
%%time
from mitnewsclassify import download

download.download()

model_2500_500_50.h5?dl=1: 0.00B [00:00, ?B/s]Package directory: /home/mbaliesnyi/anaconda3/envs/nlp/lib/python3.8/site-packages/mitnewsclassify
model_2500_500_50.h5?dl=1: 100%|██████████| 120M/120M [00:14<00:00, 8.08MB/s]
small_vocab_20.csv?dl=1: 0.00B [00:00, ?B/s]Downloading /data/tfidf/small_vocab_20.csv from https://www.dropbox.com/s/v8ytgdeumwpw4g7/small_vocab_20.csv?dl=1
small_vocab_20.csv?dl=1: 100%|██████████| 32.8k/32.8k [00:02<00:00, 15.8kB/s]
tfmer_20.p?dl=1: 0.00B [00:00, ?B/s]Downloading /data/tfidf/tfmer_20.p from https://www.dropbox.com/s/y46i525tzmf0hqx/tfmer_20.p?dl=1
tfmer_20.p?dl=1: 100%|██████████| 57.3k/57.3k [00:01<00:00, 36.1kB/s]
labelsdict_20.p?dl=1: 0.00B [00:00, ?B/s]Downloading /data/tfidf/labelsdict_20.p from https://www.dropbox.com/s/8c64ogjl82hz536/labelsdict_20.p?dl=1
labelsdict_20.p?dl=1: 100%|██████████| 16.4k/16.4k [00:01<00:00, 9.01kB/s]
nyt-theme-tags.csv?dl=1: 0.00B [00:00, ?B/s]Downloading /data/tfidf/nyt-theme-tags.csv from https://www.dropbox.c

In [3]:
# download NYT dataset
!curl https://www.dropbox.com/sh/xu9tu5hmjhuddwk/AACFtsyeBeB7mw7WW37935cYa/NYTcorpus_test.p.gz -LO
!curl https://www.dropbox.com/sh/xu9tu5hmjhuddwk/AAD31tK6oEoGlhpRZzeu3Y3Ya/NYTcorpus_train.p.gz -LO

# train and test data labels are coded in numbers,
# but the models predict human-readable labels,
# so we need to map these for model evaluation. 
# Let's use one of the files downloaded by the mitnewsclassify package
!curl https://www.dropbox.com/s/omgstbndd3xl4cy/nyt-theme-tags.csv -LO

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   172    0   172    0     0    507      0 --:--:-- --:--:-- --:--:--   507
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  1076    0  1076    0     0    592      0 --:--:--  0:00:01 --:--:--     0
100 1899M  100 1899M    0     0  8952k      0  0:03:37  0:03:37 --:--:-- 8705k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100 18902  100 18902    0     0   8290      0  0:00:02  0:00:02 --:--:-- 2050k


In [3]:
import gzip
import pickle
from random import shuffle
import requests
import csv

# open the train data given to us by Max
with gzip.open('NYTcorpus_train.p.gz', mode='r') as f:
    train_data = pickle.load(f)

# open the test data given to us by Max
with gzip.open('NYTcorpus_test.p.gz', mode='r') as f:
    test_data = pickle.load(f)

# shuffle just in case the test and train data were not shuffled before - 
# we will only measure model's accuracy on a few thousand samples
shuffle(train_data)
shuffle(test_data)

# train and test data labels are coded in numbers,
# but the models predict human-readable labels,
# so we need to re-map these. 
# Let's use one of the files downloaded by the mitnewsclassify package
with open('nyt-theme-tags.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    tags_dict = {row['tags_id']: row['tag'] for row in reader}


# extract actual article texts from data samples
train_articles = [d[2] for d in train_data] 
test_articles = [d[2] for d in test_data]

# map the number-coded labels to human-readable labels
train_labels_lists = [list(map(tags_dict.get, d[3:])) for d in train_data]
test_labels_lists = [list(map(tags_dict.get, d[3:])) for d in test_data]

In [4]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

def multi_label_scores(correct_labels, predicted_labels):

    accuracy = accuracy_score(correct_labels, predicted_labels)
    precision = precision_score(correct_labels, predicted_labels, average='weighted', zero_division=0)
    recall = recall_score(correct_labels, predicted_labels, average='weighted', zero_division=0)
    f_1_score = f1_score(correct_labels, predicted_labels, average='weighted', zero_division=0)
    
    return accuracy, precision, recall, f_1_score


In [5]:
from sklearn.preprocessing import MultiLabelBinarizer

# get the set of all 538 labels for fitting MultiLabelBinarizer
labels_flattened = []
for label in test_labels_lists:
    labels_flattened.extend(label)

label_names = list(set(labels_flattened))

mlb = MultiLabelBinarizer()
mlb.fit([sorted(label_names)])


MultiLabelBinarizer()

In [4]:
import warnings
from tqdm.autonotebook import tqdm

warnings.simplefilter(action='ignore', category=FutureWarning)


def evaluate(model, articles, labels_lists, n_of_articles = 100):

    model_name = model.__name__.split('.')[1]

    articles_subset = articles[:n_of_articles]
    labels_subset = labels_lists[:n_of_articles]

    print('predicting for', model_name)

    preds, correct = [], []

    for article, labels in tqdm(zip(articles_subset, labels_subset)):
        pred_labels = model.gettags(article)
        preds.append(pred_labels)
        correct.append(labels)

    correct, preds = mlb.transform(correct), mlb.transform(preds)
    accuracy, precision, recall, f_score = multi_label_scores(correct, preds)
    print(f'accuracy {round(accuracy,4)}, precision {round(precision,4)}, recall {round(recall,4)}, f-1 {round(f_score,4)}')
    print()
    return model_name, dict(accuracy=accuracy, precision=precision, recall=recall, f_score = f_score)


<ipython-input-4-a95a5cecddeb>:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [11]:
from mitnewsclassify import gpt2
from mitnewsclassify import tfidf
from mitnewsclassify import tfidf_bi
from mitnewsclassify import doc2vec
from mitnewsclassify import ensemble
from mitnewsclassify import trisemble
from mitnewsclassify import quadsemble
from mitnewsclassify import pentasemble

models = [
    gpt2,
    tfidf_bi,
    doc2vec,
    quadsemble,
    pentasemble,
    ensemble,
    trisemble,
    tfidf,
]

train_set_scores = {}
test_set_scores = {}

for model in models:
    print('Train: ')
    model_name, model_scores = evaluate(model, train_articles, train_labels_lists, n_of_articles=2000)
    train_set_scores[model_name] = model_scores
    
    print('Test: ')
    model_name, model_scores = evaluate(model, test_articles, test_labels_lists, n_of_articles=2000)
    test_set_scores[model_name] = model_scores


Train: 
predicting for gpt2


0it [00:00, ?it/s]

WARNING This model will consume a lot of memory, which can render your computer unusable. Please make sure that you have sufficient memory!


Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

0it [00:00, ?it/s]

accuracy 0.0, precision 0.0487, recall 0.0339, f-1 0.0291

Train: 
predicting for tfidf_bi


0it [00:00, ?it/s]

Initializing...
Model...
Count Vectorizer...
TF-IDF Transformer...
Miscellaneous...
/home/mbaliesnyi/anaconda3/envs/nlp/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.23.1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
all_train_results = sorted(train_set_scores.items(), key=lambda k: k[1]['f_score'])

all_test_results = sorted(test_set_scores.items(), key=lambda k: k[1]['f_score'])

# best model
all_train_results[-1], all_test_results[-1]

(('tfidf_bi',
  {'accuracy': 0.2955,
   'f_score': 0.5494083428664719,
   'precision': 0.7499504960230704,
   'recall': 0.462012987012987}),
 ('tfidf_bi',
  {'accuracy': 0.271,
   'f_score': 0.522029276933239,
   'precision': 0.7085726084594582,
   'recall': 0.4382817643186307}))

In [ ]:
from pprint import pprint
art = train_articles[10]
pprint(art)


('LEAD: Farm ministers of the European Community opened their first '
 'substantial debate today on a proposed freeze of guaranteed farm product '
 "prices for 1987. The bloc's executive commission is seeking to hold down "
 "prices to reduce the community's large farm surpluses. Farm ministers of the "
 'European Community opened their first substantial debate today on a proposed '
 "freeze of guaranteed farm product prices for 1987. The bloc's executive "
 "commission is seeking to hold down prices to reduce the community's large "
 'farm surpluses.')


In [ ]:
gpt2.gettags(art)

['office buildings and commercial properties',
 'marriages',
 'restaurants',
 'income tax',
 'insects',
 'abortion',
 'building (construction)']